In [1]:
#Importando la libreria para web scraping
from bs4 import BeautifulSoup
import pandas as pd
import requests
#Definimos la url de wikipedia
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#Leemos el html como texto
data = requests.get(url).text
#Leemos el html con BS
soup = BeautifulSoup(data,"html.parser")


In [2]:
#print(soup.prettify())
#Leyendo la tabla
table = soup.find("table")
dtBarToronto = pd.DataFrame(columns=['postalCode', 'borough', 'neighborhood'])
#Ciclo para las filas
for row in table.find_all("tr"):
    #Ciclo para las columnas
    for col in row.find_all("td"):
        #Condicional ignorar celdas no asignadas
        if col.find(string="Not assigned") == None:
            #Obteniendo lista de strings en la celda
            colStrings = col.p.strings
            neighborhood = ""
            #Ciclo para recorrer la lista de strings
            for i,colString in enumerate(colStrings):
                uniString = str(colString)
                #Primera posicion es el codigo postal
                if i == 0:
                    postalCode = uniString
                #Segunda posicion es el municipio
                elif i == 1:
                    borough = uniString
                else:
                    neighborhood = neighborhood + uniString

            #row_df = pd.DataFrame([postalCode,borough,])
            #df = pd.concat([row_df, df], ignore_index=True)
            neighborhood = neighborhood.strip().replace("(","").replace(")","").replace(" /",",")
            nuevaFila = {"postalCode":postalCode,"borough":borough,"neighborhood":neighborhood}
            dtBarToronto = dtBarToronto.append(nuevaFila,ignore_index=True)
            #print(postalCode,borough,neighborhood)
        else:
            print("Casilla No Asignada")


Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada
Casilla No Asignada


In [3]:
#Se asigna el valor del municipio a los barrios que se encuentran vacios
dtBarToronto['neighborhood'] = dtBarToronto['neighborhood'].fillna(dtBarToronto['borough'])

In [4]:
dtBarToronto = dtBarToronto.groupby(['postalCode','borough'])['neighborhood'].apply(','.join).reset_index()

In [5]:
#Verificamos que ninguno de los codigos postales aparece mas de una vez
dtBarToronto['postalCode'].value_counts(sort=True)


M1N    1
M9A    1
M5V    1
M7A    1
M4G    1
      ..
M2R    1
M4C    1
M4R    1
M9V    1
M5K    1
Name: postalCode, Length: 103, dtype: int64

In [6]:
dtBarToronto.shape

(103, 3)

In [8]:
"""
import geocoder # import geocoder
print("prueba print")
for index,row in dtBarToronto.iterrows():
  # initialize your variable to None
  lat_lng_coords = None
  postal_code = row["postalCode"]
  # loop until you get the coordinates
  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

  row["latitude"] = lat_lng_coords[0]
  row["longitude"] = lat_lng_coords[1]
  print(lat_lng_coords[0],lat_lng_coords[1])
"""


'\nimport geocoder # import geocoder\nprint("prueba print")\nfor index,row in dtBarToronto.iterrows():\n  # initialize your variable to None\n  lat_lng_coords = None\n  postal_code = row["postalCode"]\n  # loop until you get the coordinates\n  while(lat_lng_coords is None):\n    g = geocoder.google(\'{}, Toronto, Ontario\'.format(postal_code))\n    lat_lng_coords = g.latlng\n\n  row["latitude"] = lat_lng_coords[0]\n  row["longitude"] = lat_lng_coords[1]\n  print(lat_lng_coords[0],lat_lng_coords[1])\n'

In [10]:
df_spacial = pd.read_csv("Geospatial_Coordinates.csv")
df_spacial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
dtBarToronto.set_index("postalCode").join(df_spacial.set_index("Postal Code"))